In [1]:
# Have to specify TMPDIR and target in pip install command to work around the kernel crash issue due to 
# the small ephemeral local storage quota allocated to /tmp which is used by default by pip install
!TMPDIR=/home/hyi/temp pip install git+https://github.com/vaclab/BiasAnalyzer.git --target /home/hyi/temp --upgrade

  Cloning https://github.com/vaclab/BiasAnalyzer.git to ./temp/pip-req-build-2mkwyv9w
  Running command git clone --filter=blob:none --quiet https://github.com/vaclab/BiasAnalyzer.git /home/hyi/temp/pip-req-build-2mkwyv9w
  Resolved https://github.com/vaclab/BiasAnalyzer.git to commit a3d43525ddd2b934d8a094901f7ad62c52f2e724
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for duckdb<2.0.0,>=1.1.1 from https://files.pythonhosted.org/packages/bf/56/f627b6fcd4aa34015a15449d852ccb78d7cc6eda654aa20c1d378e99fa76/duckdb-1.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached duckdb-1.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (762 bytes)
  Obtaining dependency information for duckdb-engine<0.14.0,>=0.13.2 from https://files.pythonhosted.org/packages/5f/81/571c0373978d4e987ec2437bfb16adce6cf3b4a05761a76f1c06e859b

In [2]:
# append the target folder where HealthDataBias module was installed to PYTHONPATH
import sys
sys.path.append('/home/hyi/temp')
import pandas as pd
pd.set_option('display.max_rows', None)

In [3]:
from biasanalyzer.api import BIAS

In [4]:
# create an object of BIAS class
bias = BIAS()

no configuration file specified. Call set_config(config_file_path) next to specify configurations
Cohort Definition table created.
Cohort table created.


In [5]:
bias.set_config('/home/hyi/bias/config/config.yaml')

configuration specified in /home/hyi/bias/config/config.yaml loaded successfully


In [6]:
# the configuration file includes root_omop_cdm_database configuration info with an example shown 
# in https://github.com/hyi/HealthDataBias/blob/main/tests/assets/test_config.yaml
bias.set_root_omop()

Connected to the OMOP CDM database (read-only).


In [7]:
domains = bias.get_domains_and_vocabularies()
print(pd.DataFrame(domains))

               domain_id         vocabulary_id
0              Condition                 HCPCS
1              Condition                 ICD10
2              Condition               ICD10CM
3              Condition                ICD9CM
4              Condition                 ICDO3
5              Condition        OMOP Extension
6              Condition                SNOMED
7       Condition/Device               ICD10CM
8         Condition/Meas               ICD10CM
9       Condition Status      Condition Status
10                  Cost                  Cost
11              Currency              Currency
12                Device                 HCPCS
13                Device              ICD10PCS
14                Device                   NDC
15                Device                SNOMED
16                Device                   SPL
17                  Drug                   ATC
18                  Drug                 HCPCS
19                  Drug              ICD10PCS
20           

In [8]:
concepts = bias.get_concepts("COVID-19", "Condition", "SNOMED")
print(pd.DataFrame(concepts))

   concept_id                                       concept_name  \
0      703440  COVID-19 confirmed using clinical diagnostic c...   
1    37311061                                           COVID-19   
2    37310269                                           COVID-19   
3      703447  High risk category for developing complication...   
4      703446  Moderate risk category for developing complica...   
5      703445  Low risk category for developing complication ...   
6      703441              COVID-19 confirmed by laboratory test   

  valid_start_date valid_end_date  
0       2020-04-01     2099-12-31  
1       2020-01-31     2099-12-31  
2       2020-02-04     2020-10-28  
3       2020-04-01     2099-12-31  
4       2020-04-01     2099-12-31  
5       2020-04-01     2099-12-31  
6       2020-04-01     2099-12-31  


In [9]:
# get parent and children concept hierarchical tree for COVID-19
parent_concept_tree, children_concept_tree = bias.get_concept_hierarchy(37311061)

In [10]:
print('parent concept hierarchy for COVID-19 in text format:')
print(bias.display_concept_tree(parent_concept_tree))
print('children concept hierarchy for COVID-19 in text format:')
print(bias.display_concept_tree(children_concept_tree))

parent concept hierarchy for COVID-19 in text format:
🔼 COVID-19 (ID: 37311061, Code: 840539006)
  🔼 Clinical finding (ID: 441840, Code: 404684003)
  🔼 Viral disease (ID: 440029, Code: 34014006)
  🔼 Disease (ID: 4274025, Code: 64572001)
  🔼 Coronavirus infection (ID: 439676, Code: 186747009)
  🔼 Disease due to Coronaviridae (ID: 4100065, Code: 27619001)
  🔼 Disorder due to infection (ID: 432250, Code: 40733004)

children concept hierarchy for COVID-19 in text format:
🔽 COVID-19 (ID: 37311061, Code: 840539006)
  🔽 Lymphocytopenia due to Severe acute respiratory syndrome coronavirus 2 (ID: 3661631, Code: 866151004)
  🔽 Otitis media due to disease caused by Severe acute respiratory syndrome coronavirus 2 (ID: 37310254, Code: 1240521000000100)
  🔽 Respiratory infection caused by COVID-19 (ID: 756039, Code: OMOP4873907)
    🔽 Acute bronchitis caused by SARS-CoV-2 (ID: 3661405, Code: 138389411000119105)
    🔽 Pneumonia caused by SARS-CoV-2 (ID: 3661408, Code: 882784691000119100)
    🔽 Lower 

In [11]:
print(f'parent concept hierarchy for COVID-19 in widget tree format:')
bias.display_concept_tree(parent_concept_tree,  show_in_text_format=False)
print(f'children concept hierarchy for COVID-19 in widget tree format:')
bias.display_concept_tree(children_concept_tree, show_in_text_format=False)

parent concept hierarchy for COVID-19 in widget tree format:


children concept hierarchy for COVID-19 in widget tree format:


In [12]:
bias.cleanup()

Connection to BiasDatabase closed.
Connection to the OMOP CDM database closed.
